In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch_geometric.loader import DataLoader
from tqdm import tqdm
from graphnet import *
from configobj import ConfigObj
from Dataset import *

In [2]:
# Check if CUDA is available
cuda_available = torch.cuda.is_available()

# Set device to GPU 0 if available, otherwise CPU
device = torch.device("cuda:0" if cuda_available else "cpu")

# Print device info
print(f"CUDA available: {cuda_available}")
print(f"Using device: {device}")

CUDA available: True
Using device: cuda:0


In [3]:
#  LOAD THE DATACUBES OF THE GRID FROM C PORTA CODE

datadir = '../data_porta'
# ---- grid dimensions taken from the C code ----
nx = ny = 504
nz = 476 - 52 + 1          # 425
nlev = 6                   # caii[0] … caii[5]

# ---- memory–mapped array: reads only the chunks you touch ----
pops = np.memmap(
    f'{datadir}/AR_385_CaII_5L_pops.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, nlev)   # (k, j, i, level)==(z, y, x, L)
)

# ---- memory–mapped array: reads only the chunks you touch ----
b_xyz = np.memmap(
    f'{datadir}/AR_385_B.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 3)   # (k, j, i, B_i)==(z, y, x, B)
)

# ---- memory–mapped array: reads only the chunks you touch ----
temp = np.memmap(
    f'{datadir}/AR_385_temp.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 1)   # (k, j, i, 1)==(z, y, x, 1)
)

# ---- memory–mapped array: reads only the chunks you touch ----
vel = np.memmap(
    f'{datadir}/AR_385_veloc.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 3)   # (k, j, i, 3)==(z, y, x, 3)
)

# ---- memory–mapped array: reads only the chunks you touch ----
n_e = np.memmap(
    f'{datadir}/AR_385_ne.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 1)   # (k, j, i, 1)==(z, y, x, 1)
)

# ---- memory–mapped array: reads only the chunks you touch ----
n_p = np.memmap(
    f'{datadir}/AR_385_np.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 1)   # (k, j, i, 1)==(z, y, x, 1)
)

# ---- memory–mapped array: reads only the chunks you touch ----
n_h = np.memmap(
    f'{datadir}/AR_385_nh.dat',
    dtype='<f4',           # little-endian 32-bit float
    mode='r',
    shape=(nz, ny, nx, 1)   # (k, j, i, 1)==(z, y, x, 1)
)

# ---- memory–mapped array: reads only the chunks you touch ----
# geom = np.memmap(
#     f'{datadir}/AR_385_GEOMETRY.dat',
#     dtype='<f4',           # little-endian 32-bit float
#     mode='r',
#     shape=(nz, ny, nx, 1)   # (k, j, i, 1)==(z, y, x, 1)
# )

In [4]:
print('Populations shape:\t', pops.shape)
print('Temperature shape:\t', temp.shape)
print('Mag, field shape:\t', b_xyz.shape)
print('Velocity shape:\t\t', vel.shape)
print('N_elec shape:\t\t', n_e.shape)
print('N_nh shape:\t\t', n_h.shape)
print('N_p shape:\t\t', n_p.shape)

Populations shape:	 (425, 504, 504, 6)
Temperature shape:	 (425, 504, 504, 1)
Mag, field shape:	 (425, 504, 504, 3)
Velocity shape:		 (425, 504, 504, 3)
N_elec shape:		 (425, 504, 504, 1)
N_nh shape:		 (425, 504, 504, 1)
N_p shape:		 (425, 504, 504, 1)


In [5]:
# Read the configuration file
config_file = 'conf.dat'
with open(config_file, 'r') as f:
    tmp = f.readlines()
    f.close()

    # Parse configuration file and transform to integers
    hyperparameters = ConfigObj(tmp)

for k, q in hyperparameters.items():
    hyperparameters[k] = int(q)

# Instantiate the model with the hyperparameters
model = EncodeProcessDecode(**hyperparameters).to(device)
# Print the number of trainable parameters
print('N. total trainable parameters : {0}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))


N. total trainable parameters : 5459973


In [6]:
datast = EfficientDataset([vel, b_xyz, temp, n_h, n_e, n_p], [pops])
# Get a single sample graph
sample_graph = datast[0].to(device)

# Now, provide the input as a tuple of tensors
# NOTE: The DataLoader would create the 'batch' tensor for you. For a single graph, it's all zeros.
batch_tensor = torch.zeros(sample_graph.num_nodes, dtype=torch.long).to(device)

In [7]:
print("Model device:", next(model.parameters()).device)
print("sample_graph.x device:", sample_graph.x.device)
print("sample_graph.edge_attr device:", sample_graph.edge_attr.device)
print("sample_graph.edge_index device:", sample_graph.edge_index.device)
print("sample_graph.u device:", sample_graph.u.device)
print("batch_tensor device:", batch_tensor.device)

Model device: cuda:0
sample_graph.x device: cuda:0
sample_graph.edge_attr device: cuda:0
sample_graph.edge_index device: cuda:0
sample_graph.u device: cuda:0
batch_tensor device: cuda:0


In [8]:
print(model)

EncodeProcessDecode(
  (edge_encoder): MLP(
    (activation): ELU(alpha=1.0)
    (layers): ModuleList(
      (0): Linear(in_features=1, out_features=64, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ELU(alpha=1.0)
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (node_encoder): MLP(
    (activation): ELU(alpha=1.0)
    (layers): ModuleList(
      (0): Linear(in_features=10, out_features=64, bias=True)
      (1): ELU(alpha=1.0)
      (2): Linear(in_features=64, out_features=64, bias=True)
      (3): ELU(alpha=1.0)
      (4): Linear(in_features=64, out_features=64, bias=True)
      (5): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
  )
  (encoder_network): GraphIndependent(
    (edge_model_fn): MLP(
      (activation): ELU(alpha=1.0)
      (layers): ModuleList(
        (0): Linear(in_features=1, out_features=64, bias=True)
 

In [9]:
loader = DataLoader( datast, batch_size=64, shuffle=False)

In [10]:
lr = 1e-3
n_epochs = 10
savedir = '.'
smooth = 0.05

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Cosine annealing learning rate scheduler. This will reduce the learning rate with a cosing law
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, n_epochs)

# Loss function
loss_fn = nn.MSELoss()

# Now start the training
train_loss = []
valid_loss = []
best_loss = float('inf')

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# ... your training loop code (optimizer, model, loss, etc.)
# ... execute model(node, edge_attr, edge_index, u, batch) 


for epoch in range(1, n_epochs + 1):

    filename = str(epoch) #time.strftime("%Y%m%d-%H%M%S")

    # Compute training and validation steps
    ################### TRAINING ###################
    # Put the model in training mode
    model.train()
    print("Epoch {0}/{1}".format(epoch, n_epochs))
    t = tqdm(loader)
    loss_avg = 0.0

    for batch_idx, (data) in enumerate(t):

        # Extract the node, edges, indices, target, global and batch information from the Data class
        node = data.x
        edge_attr = data.edge_attr
        edge_index = data.edge_index
        target = data.y
        u = data.u
        batch = data.batch

        print(data)
        print(data.batch)

        # Move them to the GPU
        node, edge_attr, edge_index = node.to(device), edge_attr.to(device), edge_index.to(device)
        u, batch, target = u.to(device), batch.to(device), target.to(device)

        print(batch.shape)
        print(u.shape)
        print(batch.max())

        # Reset gradients
        optimizer.zero_grad()

        # Evaluate Graphnet
        # print(model.device)
        out = model(node, edge_attr, edge_index, u, batch)

        # Compute loss
        loss = loss_fn(out.squeeze(), target.squeeze())

        # Compute backpropagation
        loss.backward()

        # Update the parameters
        optimizer.step()

        for param_group in optimizer.param_groups:
            current_lr = param_group['lr']

        # Compute smoothed loss
        if (batch_idx == 0):
            loss_avg = loss.item()
        else:
            loss_avg = smooth * loss.item() + (1.0 - smooth) * loss_avg

    train_loss.append(loss_avg)
    ################### VALIDATION ###################
            # Do a validation of the model and return the loss

    model.eval()
    loss_avg = 0
    t = tqdm(loader)
    with torch.no_grad():
        for batch_idx, (data) in enumerate(t):

            node = data.x
            edge_attr = data.edge_attr
            edge_index = data.edge_index
            target = data.y
            u = data.u
            batch = data.batch

            node, edge_attr, edge_index = node.to(device), edge_attr.to(device), edge_index.to(device)
            u, batch, target = u.to(device), batch.to(device), target.to(device)

            out = model(node, edge_attr, edge_index, u, batch)

            loss = loss_fn(out.squeeze(), target.squeeze())

            if (batch_idx == 0):
                loss_avg = loss.item()
            else:
                loss_avg = smooth * loss.item() + (1.0 - smooth) * loss_avg

            t.set_postfix(loss=loss_avg)

    valid_loss.append(loss_avg)

    # If the validation loss improves, save the model as best
    if (valid_loss < best_loss):
        best_loss = valid_loss

        checkpoint = {
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'train_loss': train_loss,
            'valid_loss': valid_loss,
            'best_loss': best_loss,
            'hyperparameters': hyperparameters,
            'optimizer': optimizer.state_dict(),
        }

        print("Saving best model...")
        torch.save(checkpoint, savedir + filename + '_best.pth')

    # Update the learning rate
    scheduler.step()


Epoch 1/10


  0%|          | 0/3969 [00:00<?, ?it/s]

DataBatch(x=[162350, 10], y=[162350, 6], pos=[244800, 3], edge_index=[2, 4878720], edge_attr=[4878720, 1], u=[64, 1], batch=[162350], ptr=[65])
tensor([ 0,  0,  0,  ..., 63, 63, 63])
torch.Size([162350])
torch.Size([64, 1])
tensor(63, device='cuda:0')


/opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [607002,0,0], thread: [64,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [607002,0,0], thread: [65,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [607002,0,0], thread: [66,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cuda/IndexKernel.cu:92: operator(): block: [607002,0,0], thread: [67,0,0] Assertion `-sizes[i] <= index && index < sizes[i] && "index out of bounds"` failed.
/opt/conda/conda-bld/pytorch_1695392020201/work/aten/src/ATen/native/cuda/IndexKernel.cu:92:

RuntimeError: CUDA error: device-side assert triggered
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
